In [3]:
!pip install requests

In [5]:
import requests
import json
import pandas as pd

# Ripple API endpoint
url = "https://clio.altnet.rippletest.net:51234/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = response.json().get("result", {}).get("ledger_current_index")

# Define the number of ledgers to retrieve
num_ledgers = 20

# Initialize list to store ledger data
ledger_data = []

# For loop to retrieve information for the last 3 ledgers
for i in range(num_ledgers):
    ledger_index = str(int(latest_ledger_index) - i)  # Move backwards in the ledger sequence
    print(f"Retrieving Ledger Index: {ledger_index}")

    # Create JSON request payload for the ledger
    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    # Send the request
    response = requests.post(url, json=payload)

    # Process the response
    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                ledger_data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")
                })

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)

KeyboardInterrupt: 

In [9]:
import requests
import json
import pandas as pd
import concurrent.futures

# Ripple API endpoint
url = "https://testnet.xrpl-labs.com/"  # Alternative Ripple server

# First, get the latest validated ledger index
initial_payload = {
    "method": "ledger_current",
    "params": [{}]
}

# Send the request to get the latest validated ledger
response = requests.post(url, json=initial_payload)
latest_ledger_index = response.json().get("result", {}).get("ledger_current_index")

# Define the number of ledgers to retrieve
num_ledgers = 10

# Initialize list to store ledger data
ledger_data = []

# Function to retrieve ledger information
def get_ledger_data(ledger_index):
    print(f"Retrieving Ledger Index: {ledger_index}")

    payload = {
        "method": "ledger",
        "params": [
            {
                "ledger_index": ledger_index,
                "transactions": True,
                "expand": True
            }
        ]
    }

    response = requests.post(url, json=payload)

    try:
        response_json = response.json()  # Try to load the response as JSON
        ledger_info = response_json.get("result", {}).get("ledger", {})
        data = []

        # Extract information for each transaction in the ledger
        if "transactions" in ledger_info:
            for tx in ledger_info["transactions"]:
                data.append({
                    "LedgerSequence": ledger_info.get("ledger_index"),
                    "CloseTime": ledger_info.get("close_time_human"),
                    "TransactionType": tx.get("TransactionType"),
                    "Account": tx.get("Account"),
                    "Fee": tx.get("Fee"),
                    "TransactionResult": tx.get("meta", {}).get("TransactionResult"),
                    "TakerGets": tx.get("TakerGets"),
                    "TakerPays": tx.get("TakerPays"),
                    "Issuer": tx.get("TakerPays", {}).get("issuer") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Currency": tx.get("TakerPays", {}).get("currency") if isinstance(tx.get("TakerPays"), dict) else None,
                    "Sequence": tx.get("Sequence"),
                    "OfferSequence": tx.get("OfferSequence"),
                    "OwnerCount": tx.get("OwnerCount"),
                    "Flags": tx.get("Flags")
                })
        return data

    except json.JSONDecodeError:
        print(f"Error: The response for ledger {ledger_index} is not a valid JSON.")
        print("Raw response from server:")
        print(response.text)
        return []

# Use ThreadPoolExecutor to parallelize ledger retrieval
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_ledger_data, str(int(latest_ledger_index) - i)) for i in range(num_ledgers)]

    # Collect results as they complete
    for future in concurrent.futures.as_completed(futures):
        ledger_data.extend(future.result())  # Add the results to the ledger_data list

# Convert the collected data to a pandas DataFrame
df = pd.DataFrame(ledger_data)
print(df.head())


Retrieving Ledger Index: 523854
Retrieving Ledger Index: 523853
Retrieving Ledger Index: 523852
Retrieving Ledger Index: 523851
Retrieving Ledger Index: 523850
Retrieving Ledger Index: 523849
Retrieving Ledger Index: 523848
Retrieving Ledger Index: 523847
Retrieving Ledger Index: 523846
Retrieving Ledger Index: 523845
  LedgerSequence                           CloseTime TransactionType  \
0         523853  2024-Sep-07 15:39:51.000000000 UTC     NFTokenMint   
1         523853  2024-Sep-07 15:39:51.000000000 UTC     NFTokenMint   
2         523853  2024-Sep-07 15:39:51.000000000 UTC     NFTokenMint   
3         523853  2024-Sep-07 15:39:51.000000000 UTC         Payment   
4         523854                                None     NFTokenMint   

                              Account Fee TransactionResult TakerGets  \
0  rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG  10              None      None   
1  rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG  10              None      None   
2  rpU143xqysrXDNseSmpLTwNaN

In [15]:
df

,LedgerSequence,CloseTime,TransactionType,Account,Fee,TransactionResult,TakerGets,TakerPays,Issuer,Currency,Sequence,OfferSequence,OwnerCount,Flags
0,523853,2024-Sep-07 15:39:51.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461106,None,None,8
1,523853,2024-Sep-07 15:39:51.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461107,None,None,8
2,523853,2024-Sep-07 15:39:51.000000000 UTC,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461108,None,None,8
3,523853,2024-Sep-07 15:39:51.000000000 UTC,Payment,rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe,12,None,None,None,None,None,0,None,None,0
4,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461114,None,None,8
5,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461109,None,None,8
6,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461113,None,None,8
7,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461110,None,None,8
8,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461111,None,None,8
9,523854,None,NFTokenMint,rpU143xqysrXDNseSmpLTwNaN3vf4mCJMG,10,None,None,None,None,None,1461112,None,None,8


In [14]:
df.dtypes

LedgerSequence       object
CloseTime            object
TransactionType      object
Account              object
Fee                  object
TransactionResult    object
TakerGets            object
TakerPays            object
Issuer               object
Currency             object
Sequence              int64
OfferSequence        object
OwnerCount           object
Flags                 int64
dtype: object

In [16]:
# convert columns
df['LedgerSequence'] = pd.to_numeric(df['LedgerSequence'])
df['Fee'] = pd.to_numeric(df['Fee'])
df['CloseTime'] = pd.to_datetime(df['CloseTime'])

In [17]:
df.dtypes

LedgerSequence                     int64
CloseTime            datetime64[ns, UTC]
TransactionType                   object
Account                           object
Fee                                int64
TransactionResult                 object
TakerGets                         object
TakerPays                         object
Issuer                            object
Currency                          object
Sequence                           int64
OfferSequence                    float64
OwnerCount                        object
Flags                              int64
dtype: object